In [1]:
import pandas as pd

# Data Preparation

% of sand, silt, clay, and SOC were downloaded from SoilGrids of ISRIC (International Soil Reference and Information Centre) (https://soilgrids.org/) (Hengl et al., 2017). 

We Need Soil Structure Code (SSC)  and Soil Permeability Code (SPC) for K-factor calculation.;

To get SSC and SPC we need comprehensive texture class map (Raj et al. 2023). 

You can create SSC and SPC code values in ARC-MAP by referring the conditions of %sand, silt and clay using the NRCS-USDA soil texture classification system. 

first make texture class code in ARCMAP using conditions mentioned in the paper (Raj et al. 2023) 

and then get structure (s) and permeability (p) codes

In [ ]:
# Import Libraries
import numpy as np
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt
import os
import netCDF4
from osgeo import gdal
import xarray as xr

In [ ]:
# Defining all input data with variable names

sn = 'sand_india_0-30.nc'
cl = 'clay_india_0-30.nc'
si = 'silt_india_0-30.nc'
soc = 'soc_india_0-30.nc'
s1 = 'Stucture_code_Ind.nc'
p1 = 'Permeability_code_Ind.nc'

In [ ]:
# Opening All input datasets

nc1 = xr.open_dataset(s1)
nc2 = xr.open_dataset(p1)
nc3 = xr.open_dataset(sn)
nc4 = xr.open_dataset(cl)
nc5 = xr.open_dataset(si)
nc6 = xr.open_dataset(soc)


In [ ]:
#since values stored in the all datasets have variables called 'Band1'; Select to proceed further

b1 = nc1["Band1"]
b2 = nc2["Band1"]
b3 = nc3["Band1"]
b4 = nc4["Band1"]
b5 = nc5["Band1"]
b6 = nc6["Band1"]

In [ ]:
# to get the soil particle values in percentage divide SAND, SILT and CLAY by 10 and SOC by 1000, s and p are soil structure and permeability codes.

SAND = (b3/10) 
CLAY = (b4/10)
SILT = (b5/10)
SOC = (b6/100)
SOM = (SOC*2)
s = b1
p = b2

K factor using Nomograph Approach

In [ ]:
# K Factor Using Nomograph Approach

vfs = (0.2* SAND)
M = (SILT + vfs) * (100 - CLAY)


k1 = ((2.1 * M)**(1.14))*(10**(-4))*(12 - SOM)
k2 = 3.25 * (s-2)
k3 = 2.5 *(p - 3)


In [ ]:
# Final K factor using Nomograph Approach

Knomo1 = ((k1 + k2 + k3)/100)

In [ ]:
# To Save 
Knomo.to_netcdf('Knomo.nc')

K factor using EPIC Approach

In [ ]:
# Input features

SAND = (b3/10) 
CLAY = (b4/10)
SILT = (b5/10)

In [ ]:
# part of Kepic formula

A = (0.2 + (0.3 * (np.exp(-0.0256 * ((SAND * (1-(SILT/100))))))))

B1 = ((SILT/(CLAY+SILT)))
B = (B1 ** (0.3))

C = (1 - (0.25*(SOC/((SOC + np.exp((3.72 - (2.95 * SOC))))))))

SN = (1 - (SAND/100))

D = (1 - (0.7 * SN / ((SN + (np.exp((-5.51 + (22.9 * SN))))))))

In [ ]:

#Final K-factor using EPIC approach

KEPIC = (A * B * C * D)

In [ ]:

#save 

KEPIC.to_netcdf('KEPIC.nc')

# Erodibility Indices

Clay Ratio

In [ ]:
#CR

CR = (SAND+SILT)/(CLAY)

Modified Clay Ratio

In [ ]:
#MCR

MCR = (SAND+SILT)/(CLAY+SOM)

Critical Level of Organic Matter

In [ ]:
#CLOM

CLOM = (SOM)/(CLAY+SILT)

In [ ]:
# to save indices

CR.to_netcdf('CR_fin.nc')
MCR.to_netcdf('MCR_fin.nc')
CLOM.to_netcdf('CLOM_fin.nc')